# Train ASL Classifier

This notebook trains a classifier on precomputed MediaPipe hand landmarks stored in `train_landmarks.npz`.

- Features: 126-d vectors (2 hands × 21 landmarks × 3 coords)
- Labels: 29 classes (A–Z, del, nothing, space)

Tip: RBF SVM on 87k samples can be slow. You can switch to a linear model if needed.


In [ ]:
# Optional: install dependencies
# %pip install numpy scikit-learn joblib


In [3]:
import os
from typing import Tuple

import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC


In [4]:
def load_dataset(npz_path: str) -> Tuple[np.ndarray, np.ndarray]:
    data = np.load(npz_path)
    X = data["hand_landmarks"].astype(np.float32)
    y = data["labels"].astype(str)
    return X, y


def build_model() -> Pipeline:
    # Standardize features then train an SVM with RBF kernel
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(kernel="rbf", C=10.0, gamma="scale", probability=True, class_weight=None)),
    ])
    return model


In [7]:
# Parameters
DATA_PATH = "C:/Users/Nivedita/Documents/258/asl-hand-letter-detection-using-deep-learning/train_landmarks.npz"
MODEL_PATH = "C:/Users/Nivedita/Documents/258/asl-hand-letter-detection-using-deep-learning/asl_landmark_svc.joblib"
TEST_SIZE = 0.15
RANDOM_STATE = 42


In [ ]:
# Train and evaluate
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}")

X, y = load_dataset(DATA_PATH)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

model = build_model()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc:.4f}")
print("Classification report:\n")
print(classification_report(y_test, y_pred, digits=4))

joblib.dump(model, MODEL_PATH)
print(f"Saved model to {MODEL_PATH}")
